# Doc2Vec Searching of Lang Database

Use `gensim` and a simple `doc2vec` model trained on stories from the Lang coloured fairy books to support semantic retrieval of fairy stories.

The approach can be summarised as follows:

- generate a vocabulary of terms representative of the search corpus;
- generate a vector space where each dimension is a word in the vocabulary;
- generate a vector for each document or search phrase;
- retrieve documents based on similarity between document vector and search phrase vector.

The following recipe is inspired by [How to make a search engine on Movies Description](https://github.com/ppontisso/Text-Search-Engine-using-Doc2Vec-and-TF-IDF/blob/master/notebook.ipynb).

## Connecting to the Database

We're going to work with our Lang fairy story database, so let's set up a connection to it:

In [1]:
from sqlite_utils import Database

db_name = "demo.db"

db = Database(db_name)

Let's remind ourselves of the database structure:

In [2]:
print(db.schema)

CREATE TABLE [books] (
   [book] TEXT,
   [title] TEXT,
   [text] TEXT,
   [last_para] TEXT,
   [first_line] TEXT,
   [provenance] TEXT,
   [chapter_order] INTEGER,
   PRIMARY KEY ([book], [title])
);
CREATE TABLE [books_metadata] (
   [title] TEXT,
   [year] INTEGER
);
CREATE VIRTUAL TABLE [books_fts] USING FTS5 (
    [title], [text],
    content=[books]
);
CREATE TABLE 'books_fts_data'(id INTEGER PRIMARY KEY, block BLOB);
CREATE TABLE 'books_fts_idx'(segid, term, pgno, PRIMARY KEY(segid, term)) WITHOUT ROWID;
CREATE TABLE 'books_fts_docsize'(id INTEGER PRIMARY KEY, sz BLOB);
CREATE TABLE 'books_fts_config'(k PRIMARY KEY, v) WITHOUT ROWID;
CREATE TRIGGER [books_ai] AFTER INSERT ON [books] BEGIN
  INSERT INTO [books_fts] (rowid, [title], [text]) VALUES (new.rowid, new.[title], new.[text]);
END;
CREATE TRIGGER [books_ad] AFTER DELETE ON [books] BEGIN
  INSERT INTO [books_fts] ([books_fts], rowid, [title], [text]) VALUES('delete', old.rowid, old.[title], old.[text]);
END;
CREATE TRIGGER 

Recall that we can perform a full text search:

In [3]:
#q = 'king "three sons" gold'
q = 'hansel witch'
_q = f'SELECT title FROM books_fts WHERE books_fts MATCH {db.quote(q)} ;'

for row in db.query(_q):
    print(row["title"])

Hansel And Grettel


We can randomly sample a selection of rows with a query of the following form:

In [4]:
# Via https://gist.github.com/alecco/9976dab8fda8256ed403054ed0a65d7b

_q_random_sample = """
SELECT * FROM books
WHERE rowid IN (SELECT rowid FROM books
                WHERE title NOT LIKE "Preface"
                ORDER BY random() LIMIT {});
"""

for row in db.query(_q_random_sample.format(5)):
    print(row["title"])

The Fairy Nurse
The Four Gifts
The Hairy Man
The Stone-Cutter
The Little Soldier


## Simple Model

We could use an off-the-shelf model to process documents, or we can train our own model from our own documents so that the word vectors are aligned to our dataset. In a large corpus, we can train on a sample of documents if they are representative of the whole.

If we train against the whole dataset, we can search into the dataset directly from the model. If train the model on a partial collection, then we can only compare search phrases and documents that we have generated vectors for.

To create the model, it helps if we clean the documents, e.g. by decasing, and removing punctuation:

In [5]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, remove_stopwords

def clean_text(text):
    """Generate a cleaned, tokenised version of a text."""
    CUSTOM_FILTERS = [lambda x: x.lower(),
                      strip_tags, strip_punctuation,
                      strip_numeric, remove_stopwords]
    
    return preprocess_string(text, CUSTOM_FILTERS)

Apply the cleaning function to the text on the way in to creating the training corpus:

In [6]:
sample_corpus = db.query(_q_random_sample.format(9999))

sample_docs = [(clean_text(r['text']),
               f"{r['book']}::{r['title']}", #create a unique tag
               r['title'])
               for r in sample_corpus]

# For the first doc, preview the first 5 cleaned words and title
sample_docs[0][0][:5], sample_docs[0][1], sample_docs[0][2]

(['time', 'certain', 'country', 'lived', 'king'],
 'The Blue Fairy Book::The Bronze Ring',
 'The Bronze Ring')

In [7]:
# The gensim model needs named tuples
# including at least a words and tags dimension
# Naively we can just use a document index count as the tag
from collections import namedtuple

StoryDoc = namedtuple('StoryDoc',
                      'words tags title')

sample_docs_training = []

for i, sample_doc in enumerate(sample_docs):
    sample_docs_training.append(StoryDoc(sample_doc[0],
                                         [sample_doc[1]],
                                         sample_doc[2]))

In [8]:
from gensim.models import Doc2Vec

# Define the parameters for building the model.
# We can also pass a list of documents
# via the first "documents" parameter
# and the model will be trained against those.
# Alternatively, create an empty model and train it later.
model = Doc2Vec(
                # dm: training algorithm;
                # 1: distributed memory/PV-DM;
                # 0: distributed bag of words (PV-DBOW)
                dm=1,
                # vector_size: size of feature vectors
                vector_size=300,
                # window: max dist between current & predicted word
                window=10,
                # hs: 1: hierarchical softmax;
                # hs: 0 : negative sampling if negative
                hs=0,
                # min_count: ignore words w/ lower frequency
                # There is a risk to setting this too high
                # particularly if a search term is likely unique,
                # as it might be with a name. On the other hand,
                # for such situations, a simple search might be better?
                min_count=1,
                # sample: randomly downsample hi-frequency words
                # useful range: (0, 1e-5)
                sample=1e-5,
                )

The model is built around a vocabulary extracted from the training document corpus.

In [9]:
# Build the model vocabulary
model.build_vocab(sample_docs_training)

We can now train the model (this may take some time for a large corpus):

In [10]:
# It would be useful if we could display a progress bar for this
model.train(sample_docs_training,
            total_examples=model.corpus_count,
            epochs=100, start_alpha=0.01, end_alpha=0.01)

Rather than creating a model each time we want to use it, we can save the model and then load it as required:

In [11]:
# Save a model
model.save("lang_model.gensim")

# Load in a model
model = Doc2Vec.load("lang_model.gensim")

To retrieve a document matching a search phrase, we need to encode the search phrase and then try to find a matching document:

In [12]:
search_phrase = """
hansel and his sister were cast out by their wicked stepmother
and went into forest and met an evil witch
"""

# Preprocess the search phrase
tokens = clean_text(search_phrase)

tokens

['hansel',
 'sister',
 'cast',
 'wicked',
 'stepmother',
 'went',
 'forest',
 'met',
 'evil',
 'witch']

Generate a vector for the tokens:

In [13]:
# Generate the vector representation from the model
search_vector = model.infer_vector(tokens, alpha=0.001, steps = 50)

We can now search for related documents from the original training set based on how well their vectors match the vector generated for the search phrase:

In [14]:
# Find the top 10 matches
matches = model.docvecs.most_similar([search_vector], topn=10)
# To rank every document from the training corpus
# set: topn=model.docvecs.count

# The response gives the original training document ids and match scores
matches

[('The Blue Fairy Book::Hansel And Grettel', 0.7547191381454468),
 ('The Yellow Fairy Book::The Witch', 0.5689845085144043),
 ('The Red Fairy Book::The Three Dwarfs', 0.4800337553024292),
 ('The Red Fairy Book::Brother And Sister', 0.4466070830821991),
 ('The Yellow Fairy Book::The White Duck', 0.42747220396995544),
 ('The Red Fairy Book::Mother Holle', 0.4187214970588684),
 ('The Orange Fairy Book::The Two Caskets', 0.4114452302455902),
 ('The Red Fairy Book::The Twelve Brothers', 0.39679238200187683),
 ('The Orange Fairy Book::The Owl And The Eagle', 0.3924826979637146),
 ('The Yellow Fairy Book::The Nixy', 0.38102683424949646)]

Let's try another one:

In [15]:
search_phrase = """
a poor orphan girl lives with her wicked stepmother and sisters
but then her fairy godmother appears and she goes to a ball 
and leaves at midnight
but loses her slipper then finally marries the prince
"""

# Preprocess the search phrase
tokens = clean_text(search_phrase)
search_vector = model.infer_vector(tokens, alpha=0.01, steps = 50)
model.docvecs.most_similar([search_vector], topn=10)

[('The Blue Fairy Book::Cinderella, Or The Little Glass Slipper',
  0.5544741749763489),
 ('The Yellow Fairy Book::The Yellow Fairy Book', 0.5115742683410645),
 ('The Pink Fairy Book::Peter Bull', 0.5099089741706848),
 ('The Pink Fairy Book::The Merry Wives', 0.49374860525131226),
 ('The Green Fairy Book::Spindle, Shuttle, And Needle', 0.4853510856628418),
 ('The Blue Fairy Book::Rumpelstiltzkin', 0.46972209215164185),
 ('The Yellow Fairy Book::The Swineherd', 0.46485644578933716),
 ('The Yellow Fairy Book::How To Tell A True Princess', 0.46403932571411133),
 ('The Red Fairy Book::Mother Holle', 0.45142310857772827),
 ('The Blue Fairy Book::Little Red Riding Hood', 0.4508194923400879)]

Not that the result is stochastic (has a random element) in the way that the search vector is inferred: if you rerun the query, you will likely generate a different search vector. As a consequence, the search results returned are likely differ in their order and match scores each time the query is run.

## Creating a Search Tool

The next step is to register a custom SQLite function that will generate a vector for a search term and return matching records on that basis.

In [16]:
from vtfunc import TableFunction

class SQLite_doc2Vec_Model_Search(TableFunction):
    params = ['query', 'threshold']
    columns = ['book', 'title', 'score']
    name = 'doc2vec_model_search'
    
    model = Doc2Vec.load("lang_model.gensim")
    
    def initialize(self, query=None, num=None, threshold=None):
        tokens = clean_text(query)
        search_vector = model.infer_vector(tokens, alpha=0.01, steps = 50)
        scores = model.docvecs.most_similar([search_vector],
                                            topn=model.docvecs.count)
        if threshold:
            scores = [(t, s) for (t, s) in scores if s >= threshold ]

        self._iter = iter(scores)
 
    def iterate(self, idx):
        (tag, score) = next(self._iter)
        items = tag.split("::")
        return (items[0], items[1], score,)

# And register the function
SQLite_doc2Vec_Model_Search.register(db.conn)

The query searches over the model and can take various forms:

- `doc2vec_model_search("search phrase")`
- `doc2vec_model_search("search phrase", MIN_SCORE)`

For example:

In [17]:
model_query = f"""
SELECT *
FROM doc2vec_model_search('''{search_phrase}''', 0.45 );
"""

for i in db.execute(model_query):
    print(i)
 

('The Blue Fairy Book', 'Cinderella, Or The Little Glass Slipper', 0.5492352247238159)
('The Green Fairy Book', 'Spindle, Shuttle, And Needle', 0.5063244104385376)
('The Yellow Fairy Book', 'The Yellow Fairy Book', 0.4934154152870178)
('The Pink Fairy Book', 'Peter Bull', 0.48438823223114014)
('The Violet Fairy Book', 'The Child Who Came From An Egg', 0.4662984609603882)
('The Blue Fairy Book', 'Rumpelstiltzkin', 0.45463424921035767)
('The Yellow Fairy Book', 'How To Tell A True Princess', 0.45086005330085754)


## Saving Model Vectors into the Database

If we look at the object type of one of the model vectors, we see that it is a `numpy.ndarray`, which can be easily represented as a list:

In [18]:
type(model.docvecs[0])

numpy.ndarray

We can store this data in the SQLite database as a `BLOB`. To simplify the process of converting the array into and out of the appropriate format for storage in the database compared to its use as a gensim vector, we can register a custom handler for the `numpy.ndarray` object:

In [19]:
# Via https://stackoverflow.com/a/18622264/454773
# See also: https://github.com/joosephook/sqlite3-numpy
import sqlite3
import numpy as np
import io

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

Now we need to reset the database to a connection that supports the custom handler we have just registered:

In [20]:
# Reset the database connection to used the parsed datatype
db.conn = sqlite3.connect(db_name, detect_types=sqlite3.PARSE_DECLTYPES)

We can now create a table with a custom "array" datatype:

In [21]:
# Give ourselves a clean slate
db["story_vectors"].drop(ignore=True)


# sqlite_utils doesn't appear to support custom types (yet?!)
# The following errors on the "array" datatype
"""
db["story_vectors"].create({
    "book": str,
    "title": str,
    "tag": str, # a unique key derived from book and title
    "vector": "array",
}, pk=("book", "title"),
    # The following is not currently supported by sqlite_utils
   #foreign_keys=[ (("book", "title"), "books", ("book", "title"))] # local-table-id, foreign-table, foreign-table-id]
)
"""

# so we can create a table the old fashioned way...
vector_table_create = """
CREATE TABLE story_vectors 
    (tag TEXT PRIMARY KEY, vector array, book TEXT, title TEXT );
"""

cur = db.conn.cursor()
cur.execute(vector_table_create)

We can generate a list of dictionaries, one per record used to train the model, that can then be added directly to the `story_vectors` database table:

In [22]:
xrecords = []

for tag in model.docvecs.doctags:
    xrecords.append({'book': tag.split('::')[0],
                     'title': tag.split('::')[1],
                     'tag': tag, 'vector':model.docvecs[tag]})

And add the records directly to the database:

In [23]:
db["story_vectors"].insert_all(xrecords)

<Table story_vectors (tag, vector, book, title)>

Let's pull an example record back showing just the first few elements of the vector associated with the record:

In [24]:
_q = f'SELECT * FROM story_vectors LIMIT 1;'

for row in db.query(_q):
    print(row['tag'], row['book'], row['title'], row['vector'][:10])

The Blue Fairy Book::The Bronze Ring The Blue Fairy Book The Bronze Ring [-0.01405123 -0.50704396  0.7826763   0.6966156   0.87891227 -0.2688267
  0.33178896 -1.6542805   1.053172    0.8667782 ]


In [25]:
import pandas as pd

_q = f'SELECT * FROM story_vectors;'
df = pd.read_sql(_q, db.conn)

df.head(3)

,tag,vector,book,title
0,The Blue Fairy Book::The Bronze Ring,"[-0.014051229, -0.50704396, 0.7826763, 0.69661...",The Blue Fairy Book,The Bronze Ring
1,The Blue Fairy Book::Prince Hyacinth And The D...,"[0.22857311, -0.0002901034, 0.86985964, -0.280...",The Blue Fairy Book,Prince Hyacinth And The Dear Little Princess
2,The Blue Fairy Book::East Of The Sun And West ...,"[0.12482449, 0.509038, 0.730183, -1.0778545, 0...",The Blue Fairy Book,East Of The Sun And West Of The Moon


We can get the cosine similarity for each row relative to a search vector using the `sklearn.metrics.pairwise.cosine_similarity` applied to a dataframe of vectors we want to match against.

The `cosine_similarity()` function will happily accept two `pandas` dataframes, such as an N x M matrix of vectors we want to score against, and a 1 x M search vector matrics:

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

# Grab the vectors and expand each on across columns
match_vectors_df = df['vector'].apply(pd.Series)
search_vector_df = pd.DataFrame(search_vector).T

df['score'] = cosine_similarity(match_vectors_df,
                                search_vector_df)

df[df['score']>0.45].sort_values("score", ascending=False).head()

,tag,vector,book,title,score
6,"The Blue Fairy Book::Cinderella, Or The Little...","[1.1064563, 1.6556664, 0.65659255, 0.12633619,...",The Blue Fairy Book,"Cinderella, Or The Little Glass Slipper",0.554474
275,The Yellow Fairy Book::The Yellow Fairy Book,"[0.2586239, 0.2307802, 0.54648733, -0.15965664...",The Yellow Fairy Book,The Yellow Fairy Book,0.511574
51,The Pink Fairy Book::Peter Bull,"[-1.1498377, 0.6431778, 0.94959325, 0.12849873...",The Pink Fairy Book,Peter Bull,0.509909
71,The Pink Fairy Book::The Merry Wives,"[-0.4850009, 1.2584103, 0.38522613, 0.6031495,...",The Pink Fairy Book,The Merry Wives,0.493749
389,"The Green Fairy Book::Spindle, Shuttle, And Ne...","[-0.0354758, 0.32155505, 0.37913308, -0.182975...",The Green Fairy Book,"Spindle, Shuttle, And Needle",0.485351


So it's easy enough to create a custom function to search over the vectors table rather than the model:

In [27]:
class SQLite_doc2Vec_Table_Search(TableFunction):
    params = ['query', 'threshold']
    columns = ['book', 'title', 'score']
    name = 'doc2vec_search'
    
    # If we move this into the body, we can update the database
    # and respond to new rows added to story_vectors table
    _q = f'SELECT book, title, vector FROM story_vectors;'
    df = pd.read_sql(_q, db.conn)
    match_vectors_df = df['vector'].apply(pd.Series)
    
    def initialize(self, query=None, threshold=None):
        df = self.df
        tokens = clean_text(query)
        search_vector = model.infer_vector(tokens, alpha=0.01, steps = 50)

        search_vector_df = pd.DataFrame(search_vector).T

        # Find cosine similarity
        df['score'] = cosine_similarity(self.match_vectors_df,
                                        search_vector_df)
        
        # Apply minimum threshold if set
        _iterator = df[df['score']>=threshold] if threshold else df

        self._iter = _iterator[self.columns].itertuples(index=False,
                                                        name=None)
 
    def iterate(self, idx):
        row = next(self._iter)
        if row:
            return (row[0],row[1], row[2],)

# And register the function
SQLite_doc2Vec_Table_Search.register(db.conn)

Let's try it out:

In [28]:
vector_query = f"""
SELECT *
FROM doc2vec_search('''{search_phrase}''')
WHERE score>0.4 ORDER BY score DESC LIMIT 3;
"""

pd.read_sql(vector_query, db.conn)

,book,title,score
0,The Blue Fairy Book,"Cinderella, Or The Little Glass Slipper",0.524345
1,The Green Fairy Book,"Spindle, Shuttle, And Needle",0.500835
2,The Yellow Fairy Book,The Yellow Fairy Book,0.488937
